# Boto3 SDK를 통해 Amazon Bedrock agent 생성 및 호출

> *이 노트북은 SageMaker Studio의 **Data Science 3.0`** 커널과 잘 작동해야 합니다.*

## 소개

이 노트북은 `bedrock-agent` 및 `bedrock-agent-runtime` boto3 클라이언트를 사용하여 다음을 수행하는 방법을 보여줍니다.
- 에이전트 생성
- 작업 그룹 생성
- 에이전트를 작업 그룹과 연결하고 사용할 수 있도록 준비
- 에이전트 별칭 생성
- 에이전트 호출

우리는 Boto3 API를 통해 Bedrock의 Claude v3 Sonnet을 활용할 것입니다.

**Note:** *이 노트북은 AWS 환경 내부와 외부에서 모두 실행되도록 설계되었습니다.*

### 필수 조건

다음 권한이 있는 AWS 계정이 있는지 확인하세요.
- IAM 역할 및 정책 생성 및 관리
- AWS Lambda 함수 생성 및 호출
- Amazon S3 버킷 생성, 읽기 및 쓰기
- Amazon Bedrock 에이전트 및 모델 액세스 및 관리
- Amazon Glue 데이터베이스 및 크롤러 생성 및 관리
- 쿼리 실행 및 Amazon Athena 작업 공간 관리

### Context

다음 섹션에서는 Boto3 SDK를 사용하여 Bedrock 에이전트를 만들고 호출하는 방법을 안내합니다.

### 사용 사례

노트북은 자연어 질문에서 SQL 쿼리를 작성할 수 있는 에이전트를 설정합니다. 
그런 다음 데이터베이스에서 응답을 검색하여 사용자 문의에 정확한 답변을 제공합니다. 
아래 다이어그램은 이 솔루션의 아키텍처를 간략하게 설명합니다

![sequence-flow-agent](images/text-to-sql-architecture-Athena.png)

에이전트는 다음을 위해 설계되었습니다.
- 데이터베이스 스키마 검색
- SQL 쿼리 실행


In [ ]:
upgrade_output = !pip install --upgrade pip
install_boto3_output = !pip install boto3
##Ensure your boto3 and botocore libraries are up to date
upgrade_output_botocore_boto3= !pip install --upgrade boto3 botocore 

In [ ]:
from dependencies.config import *

사전에 필요한 인프라를 구성합니다.  (IAM Role, Policy 생성 및 맵핑, 람다 함수 등록 )

In [ ]:
!python ./dependencies/build_infrastructure.py

이 코드는 AWS Bedrock Agent의 특정 Agent와 그에 해당하는 Alias를 찾아 식별자를 추출하는 작업을 수행합니다.

1. 의존성 임포트 
2. Agent 목록 조회
3. Agent ID 찾기
4. Agent Alias 목록 조회
- 찾은 agent_id에 대한 모든 alias 목록을 조회합니다.
5. Agent Alias ID 찾기
- 응답에서 특정 agent_alias_name과 일치하는 alias의 ID를 찾습니다.
- 찾은 ID를 agent_alias_id 변수에 저장합니다.

In [ ]:
from dependencies.config import *

list_agent=bedrock_agent_client.list_agents()['agentSummaries']
#print(list_agent)
#print(agent_name)
# Search f
agent_id = next((agent['agentId'] for agent in list_agent if agent['agentName'] == agent_name), None)

print(agent_id)

response = bedrock_agent_client.list_agent_aliases(
    agentId=agent_id,
)
response['agentAliasSummaries']
agentAliasId=next((agent['agentAliasId'] for agent in response['agentAliasSummaries'] if agent['agentAliasName'] == agent_alias_name), None)
agent_alias_id=agentAliasId
print(agent_alias_id)

에이전트 호출

이제 에이전트를 만들었으니 `bedrock-agent-runtime` 클라이언트를 사용하여 이 에이전트를 호출하고 몇 가지 작업을 수행해 보겠습니다.

In [ ]:

query_to_agent = """What was John Denny's salary in 1986?"""

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False


# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText=query_to_agent,
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)
logger.info(pprint.pprint(agentResponse))

AWS Bedrock Agent 응답 스트리밍 처리

1. Agent 응답 스트림 처리 시작
   event_stream = agentResponse['completion']

2. 스트림 이벤트 처리
   try-except 구조로 예외 상황 대비
   이벤트 타입별 처리:
   - chunk: 실제 응답 데이터 (UTF-8 디코딩)
   - trace: 디버깅 정보
   - 기타: 예상치 못한 이벤트로 예외 처리

In [ ]:

%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            #logger.info(json.dumps(event['trace'], indent=2))
            logger.info(event)
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
# 그리고 에이전트의 답변만 보고 싶다면 여기에 답변이 있습니다.
print(agent_answer)

에이전트를 호출하는 함수 생성

In [ ]:

def invoke_agent(query):
    ## create a random id for session initiator id
    session_id:str = str(uuid.uuid1())
    enable_trace:bool = False
    end_session:bool = False
 

    # 에이전트 API를 호출합니다
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    event_stream = agentResponse['completion']
    print("Fetching answer...")
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)



1. 함수 목적
- Bedrock Agent에 질의를 보내고 응답을 받는 함수
- 특정 질문(Nolan Ryan의 명예의 전당 입성 연도)에 대한 답변 요청

2. 매개변수
- 문자열 형태의 질문을 인자로 전달
- 여기서는 "What year was Nolan Ryan inducted into the Hall of Fame?"

3. 예상 동작
- Agent에 질문을 전송
- Agent가 데이터베이스/지식베이스를 검색
- 관련 정보를 찾아 응답 생성
- 응답을 스트리밍 형태로 반환

4. 사용 목적
- 야구 관련 정보 조회
- 명예의 전당 입성 년도와 같은 구체적 정보 획득

In [ ]:
invoke_agent("What year was Nolan Ryan inducted into the Hall of Fame?")

invoke_agent() 함수 호출
Purpose: MLB 명예의 전당 관련 질의 처리
Parameter: "What year was Nolan Ryan inducted into the Hall of Fame?"
기대값: Nolan Ryan의 명예의 전당 입성 연도 반환

Note:
- 질문은 명확한 단일 답변이 있는 형태
- 영어로 질의/응답 처리
- 야구 도메인 특화 질문

In [ ]:
invoke_agent("What year was Nolan Ryan inducted into the Hall of Fame?")


In [ ]:
invoke_agent("In what year did Hank Aaron hit the most home runs?")


이 쿼리는 질문에 답하기 위해 두 테이블의 조인이 필요합니다.

In [ ]:

invoke_agent("What player has received the most All-Star Game selections?")

이 쿼리의 응답은 올해에 사용 가능한 데이터가 없다고 말해야 합니다!!

In [ ]:

invoke_agent("What was Babe Ruth's salary in 1930?")


In [ ]:
invoke_agent("Who is the richest player in baseball history? ")


In [ ]:
invoke_agent("What was John Denny's salary in 1986? ")


In [ ]:
# 이렇게 하면 노트북 실행이 중지되고 사용자가 Enter 키를 누를 때까지 기다립니다.
input("Press Enter to continue...")


## Clean Up - 리소스 정리 (선택 사항)

다음 단계는 선택 사항이며, 우리가 만든 인프라(AWS 리소스)를 삭제합니다.



In [ ]:
!python ./dependencies/clean.py


## 결론
이제 `boto3` SDK를 사용하여 에이전트를 생성, 호출 및 삭제하는 방법을 실험했습니다.

### 요점
- 이 노트북을 응용 프로그램에 대한 새 에이전트를 생성하도록 조정

## 감사합니다